In [55]:
import argparse
import torch
import numpy as np
from torch import nn
from pathlib import Path
import slayerSNN as snn

In [56]:
class FLAGS():
    def __init__(self):
        self.path = '/home/tasbolat/some_python_examples/data_VT_SNN/'
        self.theta = 10
        self.tauRho = 1
        self.tsample= 325
        self.count=1
args = FLAGS()

In [57]:
device = torch.device('cuda:1')

In [58]:
params = {
    "neuron": {
        "type": "SRMALPHA",
        "theta": args.theta,
        "tauSr": 10.0,
        "tauRef": 1.0,
        "scaleRef": 2,
        "tauRho": args.tauRho, # pdf
        "scaleRho": 1,
    },
    "simulation": {"Ts": 1.0, "tSample": args.tsample, "nSample": 1}
}

In [59]:
# define average pooling for vision
class SimplePool(nn.Module):

    def __init__(self, netParams):
        super(SimplePool, self).__init__()
        self.slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.pool = self.slayer.pool(4)

    def forward(self, input_data):
        spike_out = self.slayer.spike(self.pool(self.slayer.psp(input_data)))
        return spike_out

In [60]:
net = SimplePool(params).to(device)

In [46]:
#net.pool.weight = torch.nn.Parameter(net.pool.weight/4, requires_grad=False)

In [61]:
#net.pool.weight

In [62]:
for i in range(args.count):
   
    print(f"Processing vision with pooling {i}...")
    vis_npy = Path(args.path) / f"{i}_vis.npy"
    vis = torch.FloatTensor(np.load(vis_npy)).unsqueeze(0)
    vis = vis.to(device)
    #vis = vis.permute(0,1,4,2,3)
    
    print(vis.shape)
   
    with torch.no_grad():
        vis_pooled = net.forward(vis)
        #vis_pooled = vis_pooled.squeeze().permute(0,2,3,1)
       
    #ds_vis_arr.append(vis_pooled.detach().cpu().squeeze(0))

Processing vision with pooling 0...
torch.Size([1, 2, 250, 200, 325])


In [63]:
vis_pooled.shape

torch.Size([1, 2, 63, 50, 325])

In [64]:
for p in net.parameters():
    print(p)

Parameter containing:
tensor([[[[[11.],
           [11.],
           [11.],
           [11.]],

          [[11.],
           [11.],
           [11.],
           [11.]],

          [[11.],
           [11.],
           [11.],
           [11.]],

          [[11.],
           [11.],
           [11.],
           [11.]]]]], device='cuda:1')


In [65]:
### Image demonstrate

In [66]:
from IPython.display import HTML

In [67]:
anim = snn.io.animTD(snn.io.spikeArrayToEvent(vis.squeeze().cpu().numpy()), frameRate=27)
HTML(anim.to_jshtml())

In [68]:
anim = snn.io.animTD(snn.io.spikeArrayToEvent(vis_pooled.squeeze().cpu().numpy()), frameRate=27)
HTML(anim.to_jshtml())